# This notebook is for preparing the data for SEM analysis

In [ ]:
#1. labeling all the files for visualization purposes. This section can be skipped

import pandas as pd 
import pandas as pd 
import os
import datetime as dt

#preparing annotation file

def get_sec(time_str):
    """Get Seconds from time."""
    #print(time_str)
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)


def time_transfer(dateandtime,video_name):
    base_time = dt.datetime.strptime(video_name[:4]+"-"+video_name[4:6]+"-"+video_name[6:8]+"T"+video_name.split("_")[1][:2]+":"+video_name.split("_")[1][2:4]+":"+video_name.split("_")[1][4:6]+".000","%Y-%m-%dT%H:%M:%S.%f")-dt.timedelta(hours=4)#.microsecond
    item_time = dt.datetime.strptime(dateandtime.split("+")[0],"%Y-%m-%dT%H:%M:%S.%f")
    timestamp = ((item_time-base_time).seconds+(item_time-base_time).microseconds/1000000)-36000
    return timestamp
gt = pd.read_csv("H:/19/From Work/From Work.csv")
#gt = gt[gt["Video"] == "20200203_164117_NR.MP4"] 
gt["time_start"] = (gt["Video"].str[:-7])
gt["time_start"] = pd.to_datetime(gt["time_start"],format = '%Y%m%d_%H%M%S')
gt["Timestamp Begin"] = gt["Timestamp Begin"].apply(get_sec)
gt["Timestamp End"] = gt["Timestamp End"].apply(get_sec)
gt["Timestamp Begin"] = pd.to_timedelta(gt["Timestamp Begin"],unit='s')
gt["Timestamp End"] = pd.to_timedelta(gt["Timestamp End"],unit='s')
gt["current_time_start"] = gt["time_start"] + gt["Timestamp Begin"] 
gt["current_time_start"] = gt["current_time_start"].dt.tz_localize(None)
gt["current_time_end"] = gt["time_start"] + gt["Timestamp End"] 
gt["current_time_end"] = gt["current_time_end"].dt.tz_localize(None)

gt_inside = gt[gt["Video"].str.endswith("R.MP4")]
gt_outside = gt[gt["Video"].str.endswith("F.MP4")]
gt_outside = gt_outside[gt_outside["Event"] != "driving in a city street"]
gt_inside = gt_inside[gt_inside["Event"] != "talking on phone without"]


target_file = acc.copy()
target_file["Timestamp_new"] = pd.to_datetime(target_file["Timestamp"]) + pd.Timedelta(value=-5,unit='h')
target_file["Timestamp_new"] = target_file["Timestamp_new"].dt.tz_localize(None)
target_file["event_inside"] ="none"
target_file["event_outside"] ="none"
target_file["timestamp_inside"] = "none"
target_file["timestamp_outside"] = "none"
target_file["source_inside"] = "none"
target_file["source_outside"] = "none"
for index,row in gt_inside.iterrows():
    target_file["event_inside"][(target_file['Timestamp_new']>(row['current_time_start'])) 
                                & (target_file['Timestamp_new']<(row['current_time_end']))] = row["Event"]
    target_file["timestamp_inside"][(target_file['Timestamp_new']>(row['current_time_start'])) 
                                & (target_file['Timestamp_new']<(row['current_time_end']))] = row["Timestamp Begin"]
    target_file["source_inside"][(target_file['Timestamp_new']>(row['current_time_start'])) 
                                & (target_file['Timestamp_new']<(row['current_time_end']))] = row["Video"]
for index,row in gt_outside.iterrows():
    target_file["event_outside"][(target_file['Timestamp_new']>(row['current_time_start'])) 
                                & (target_file['Timestamp_new']<(row['current_time_end']))] = row["Event"]
    target_file["timestamp_outside"][(target_file['Timestamp_new']>(row['current_time_start'])) 
                                & (target_file['Timestamp_new']<(row['current_time_end']))] = row["Timestamp Begin"]
    target_file["source_outside"][(target_file['Timestamp_new']>(row['current_time_start'])) 
                                & (target_file['Timestamp_new']<(row['current_time_end']))] = row["Video"]
#target_file.to_csv(file[:-4]+"_labeled"+".csv")

In [ ]:
#2.1 Reading all sources of wearable, vehicle, and gaze data

#read car's data
#reading acc,gyro, and linear acc data, put them together, resample at 100 hz, fill nans and make it a clean dataframe
import pandas as pd 
from tqdm import tqdm

sensors = ["GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum"]
df_ultimate = []

for sensor in sensors:
    df_temp = pd.read_csv("C:/Users/Arsalan/Desktop/CVILLE - DC Trips/both hands and car - DC - CVille/raw/car/"+"Smartwatch_"+sensor+".csv",parse_dates=['Timestamp'])
    #start_date = "2020-08-21T23:30:59.154+0000"
    #end_date = "2020-08-22T02:14:56.307+0000"
    start_date = "2020-08-23T15:16:11.000+0000"
    end_date = "2020-08-23T17:16:00.000+0000"
    mask = (df_temp['Timestamp'] > start_date) & (df_temp['Timestamp'] <= end_date)
    df_temp = df_temp.loc[mask]
    if sensor in ["GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum"]:
        df_temp.rename(columns={'X': 'X'+sensor[:4], 'Y': 'Y'+sensor[:4],'Z':'Z'+sensor[:4]}, inplace=True)
    df_ultimate.append(df_temp)

    
df_final = df_ultimate[0]
for df in df_ultimate[1:]:
    df = df.drop(columns=["participantid"])
    df.reset_index(drop=True,inplace=True)
    df_final = df_final.merge(df,on=['Timestamp',
                                         'DeviceId'], how='outer')
    
    
df_final = df_final.set_index("Timestamp")
df_final = df_final.groupby(["DeviceId"]).resample('100L').mean().ffill().bfill()
df_final = df_final.reset_index()
df_final["magGyro"] = (df_final["XGyro"]**2 + df_final["YGyro"]**2 + df_final["ZGyro"]**2)**0.5
df_final["magAcce"] = (df_final["XAcce"]**2 + df_final["YAcce"]**2 + df_final["ZAcce"]**2)**0.5
df_final["magLine"] = (df_final["XLine"]**2 + df_final["YLine"]**2 + df_final["ZLine"]**2)**0.5

#keep a copy to not rerun things
imu_data_car = df_final.copy()
imu_data_car = imu_data_car.set_index("Timestamp")
imu_data_car = df_final.copy()
imu_data_car = imu_data_car.set_index("Timestamp")
imu_data_car = imu_data_car[1:]

In [ ]:
#2.2 Reading all sources of wearable, vehicle, and gaze data
#reading acc,gyro, and linear acc data, put them together, resample at 100 hz, fill nans and make it a clean dataframe
import pandas as pd 
from tqdm import tqdm

sensors = ["HeartRateDatum","GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum","Light"]
df_ultimate = []

for sensor in sensors:
    df_temp = pd.read_csv("C:/Users/Arsalan/Desktop/CVILLE - DC Trips/both hands and car - DC - CVille/raw/"+"Smartwatch_"+sensor+".csv",parse_dates=['Timestamp'])
#     start_date = "2020-08-21T23:30:59.154+0000"
#     end_date = "2020-08-22T02:14:56.307+0000"
    start_date = "2020-08-23T15:16:11.000+0000"
    end_date = "2020-08-23T17:16:00.000+0000"
    mask = (df_temp['Timestamp'] > start_date) & (df_temp['Timestamp'] <= end_date)
    df_temp = df_temp.loc[mask]
    if sensor in ["GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum"]:
        df_temp.rename(columns={'X': 'X'+sensor[:4]+"_driver", 'Y': 'Y'+sensor[:4]+"_driver",'Z':'Z'+sensor[:4]+"_driver"}, inplace=True)
    df_ultimate.append(df_temp)

    
df_final = df_ultimate[0]
for df in df_ultimate[1:]:
    df = df.drop(columns=["participantid"])
    df.reset_index(drop=True,inplace=True)
    df_final = df_final.merge(df,on=['Timestamp',
                                         'DeviceId'], how='outer')
    
    
df_final = df_final.set_index("Timestamp")
df_final = df_final.groupby(["DeviceId"]).resample('100L').mean().ffill().bfill()
df_final = df_final.reset_index()
df_final["magGyro_driver"] = (df_final["XGyro_driver"]**2 + df_final["YGyro_driver"]**2 + df_final["ZGyro_driver"]**2)**0.5
df_final["magAcce_driver"] = (df_final["XAcce_driver"]**2 + df_final["YAcce_driver"]**2 + df_final["ZAcce_driver"]**2)**0.5
df_final["magLine_driver"] = (df_final["XLine_driver"]**2 + df_final["YLine_driver"]**2 + df_final["ZLine_driver"]**2)**0.5

#keep a copy to not rerun things
hr_imu_data = df_final.copy()
hr_imu_data = hr_imu_data.set_index("Timestamp")
hr_imu_data_left=hr_imu_data[hr_imu_data["DeviceId"]=="f60691a313420a4e"]
hr_imu_data_right=hr_imu_data[hr_imu_data["DeviceId"]=="39ca51c16b9ec429"]
hr_imu_data_left.columns = [str(col) + '_left' for col in hr_imu_data_left.columns]
#imu_data_right = imu_data_right.set_index("Timestamp")
#imu_data_left = imu_data_left.set_index("Timestamp")

In [ ]:
#2.3 Reading all sources of wearable, vehicle, and gaze data
#reading gaze data

cols = [ " AU01_r"," AU02_r"," AU04_r"," AU05_r"," AU06_r"," AU07_r"," AU09_r"," AU10_r"," AU12_r"," AU14_r"," AU15_r"," AU17_r"," AU20_r"," AU23_r"," AU25_r"," AU26_r"," AU45_r"," AU01_c"," AU02_c"," AU04_c"," AU05_c"," AU06_c"," AU07_c"," AU09_c"," AU10_c"," AU12_c"," AU14_c"," AU15_c"," AU17_c"," AU20_c"," AU23_c"," AU25_c"," AU26_c"," AU28_c", " AU45_c"
," timestamp"," confidence"," success"," gaze_angle_x"," gaze_angle_y"," p_scale"," p_rx"," p_ry"," p_rz"," p_tx"," p_ty","video_name","day" ]
gaze = pd.read_csv("C:/Users/Arsalan/Desktop/CVILLE - DC Trips/both hands and car - DC - CVille/raw/final_gaze.csv",usecols=cols)

gaze["time_start"] = (gaze["video_name"].str[:-7])
gaze["time_start"] = pd.to_datetime(gaze["time_start"],format = '%Y%m%d_%H%M%S')
gaze[" timestamp"] = pd.to_timedelta(gaze[" timestamp"],unit = 's')
gaze["Timestamp"] = gaze["time_start"] + gaze[" timestamp"]
gaze["mag_gaze"] = (gaze[" gaze_angle_x"]**2 + gaze[" gaze_angle_y"]**2)**0.5
gaze["Timestamp"] = gaze["Timestamp"] + pd.Timedelta(value=+4,unit='h')
# start_date = pd.Timestamp("2020-08-21T23:30:59.154+0000",tz=None)
# end_date = pd.Timestamp("2020-08-22T02:14:56.307+0000",tz=None)
start_date = pd.Timestamp("2020-08-23T15:16:11.000+0000",tz=None)
end_date = pd.Timestamp("2020-08-23T17:16:00.000+0000",tz=None)
start_date = start_date.tz_localize(None)
end_date = end_date.tz_localize(None)
gaze["Timestamp"] = pd.to_datetime(gaze["Timestamp"])  
gaze["Timestamp"] = gaze["Timestamp"].dt.tz_localize(None)
mask = (gaze["Timestamp"] > start_date) & (gaze["Timestamp"] <= end_date)
gaze = gaze.loc[mask]
gaze = gaze.set_index("Timestamp")
gaze = gaze[gaze[" success"] == 1]
gaze = gaze[gaze[" confidence"]>0.85]
gaze = gaze.resample('100L').mean().bfill().ffill()
gaze = gaze.drop(columns=['day'])

In [ ]:
#2.4 getting all the outside environment data

import os
import pandas as pd

cols = ["person","bicycle","car","motorcycle","bus","truck","frame","follow_dist","follow_obj","lead_pres","video_name","day" ]
mask_rcnn = pd.read_csv("D:/Google Drive/UVA_PhD/Projects/Wearable Projects/Codes/Wearable_Event_detection/csv files/final_mask_rcnn.csv",usecols=cols)

mask_rcnn["time_start"] = (mask_rcnn["video_name"].str[:-16])
mask_rcnn["time_start"] = pd.to_datetime(mask_rcnn["time_start"],format = '%Y%m%d_%H%M%S')
mask_rcnn[" timestamp"] = mask_rcnn["frame"]*0.0333
mask_rcnn[" timestamp"] = pd.to_timedelta(mask_rcnn[" timestamp"],unit = 's')
mask_rcnn["Timestamp"] = mask_rcnn["time_start"] + mask_rcnn[" timestamp"]
mask_rcnn["Timestamp"] = mask_rcnn["Timestamp"] + pd.Timedelta(value=+4,unit='h')
# start_date = pd.Timestamp("2020-08-21T23:30:59.154+0000",tz=None)
# end_date = pd.Timestamp("2020-08-22T02:14:56.307+0000",tz=None)
start_date = pd.Timestamp("2020-08-23T15:16:11.000+0000",tz=None)
end_date = pd.Timestamp("2020-08-23T17:16:00.000+0000",tz=None)
start_date = start_date.tz_localize(None)
end_date = end_date.tz_localize(None)
mask_rcnn["Timestamp"] = pd.to_datetime(mask_rcnn["Timestamp"])  
mask_rcnn["Timestamp"] = mask_rcnn["Timestamp"].dt.tz_localize(None)
mask = (mask_rcnn["Timestamp"] > start_date) & (mask_rcnn["Timestamp"] <= end_date)
mask_rcnn = mask_rcnn.loc[mask]
mask_rcnn = mask_rcnn.set_index("Timestamp")
mask_rcnn["lead_pres"] = mask_rcnn["lead_pres"].fillna(0)
mask_rcnn["follow_obj"] = mask_rcnn["follow_obj"].fillna(15)
mask_rcnn["follow_dist"] = mask_rcnn["follow_dist"].fillna(5000)
mask_rcnn = mask_rcnn.resample('100L').bfill().ffill()
    

In [ ]:
#3. put them all together

hr_imu_data_right.index = hr_imu_data_right.index.tz_localize(None)
hr_imu_data_left.index = hr_imu_data_left.index.tz_localize(None)
imu_data_car.index = imu_data_car.index.tz_localize(None)
total_data = pd.concat([mask_rcnn,gaze,hr_imu_data_right,hr_imu_data_left,imu_data_car],axis=1)

#4. combining them with change points - copying bcp groups from previous steps
total_data = total_data[:-1]
#total_data["bcp_groups"] = list_groups["new_group"].values
#total_data = total_data.dropna()
total_data["gaze_std_x"]=total_data[" gaze_angle_x"].rolling(10,min_periods=0).std()
total_data["gaze_std_x"] = total_data["gaze_std_x"].bfill()
total_data["gaze_std_y"]=total_data[" gaze_angle_y"].rolling(10,min_periods=0).std()
total_data["gaze_std_y"] = total_data["gaze_std_y"].bfill()
total_data = total_data.bfill().ffill()
#total_data.to_csv("data_coming_back_100L.csv")

In [ ]:
#4. exploring different discreetizations

import plotly.express as px
from sklearn import mixture
import matplotlib.pyplot as plt
import plotly.express as px

data_all = total_data.copy()
#data = data_all.drop([' confidence',' success','Timestamp','DeviceId','DeviceId.1','video_name'],axis=1)
for col in data_all.columns:
    if col in ['gaze_std','follow_dist', ' gaze_angle_x', ' gaze_angle_y',
       ' p_scale', ' p_rx', ' p_ry', ' p_rz', ' p_tx', ' p_ty', 'mag_gaze',
       'HR', 'XGyro_driver', 'YGyro_driver', 'ZGyro_driver', 'XAcce_driver',
       'YAcce_driver', 'ZAcce_driver', 'XLine_driver', 'YLine_driver',
       'ZLine_driver', 'Light', 'magGyro_driver', 'magAcce_driver',
       'magLine_driver', 'XGyro', 'YGyro', 'ZGyro', 'XAcce', 'YAcce', 'ZAcce',
       'XLine', 'YLine', 'ZLine', 'magGyro', 'magAcce', 'magLine']:
        if col=="mag_gaze":
            print(col)
            temp = data_all[col].copy()
            X = temp.values
            X = X.reshape(-1,1)
            labels = (mixture.GaussianMixture(n_components=2).fit(X)).predict(X)
            data_all[(col+str("_label"))] = labels
data_all = data_all.reset_index()
fig = px.scatter(data_all, x="Timestamp", y="mag_gaze",color= "mag_gaze_label",labels={
                     "Timestamp": "<b>Time",
                     "HR": "<b>HR (bpm)",
                 })

fig.update_layout(    
        {   
            'yaxis':{'dtick': 10,'title':"<b>HR (bpm)",'linecolor':'black','linewidth':10},
            'xaxis':{'title':"<b>Time",'linecolor':'black','linewidth':10},
            'height':600,
            'width':1200,
            'font':dict(size=24),
            'paper_bgcolor':'rgba(0,0,0,0)',
            'plot_bgcolor':'rgba(0,0,0,0)'
        }
    )

fig.show()

In [ ]:
#5. reading all the data that is saved from the previous sections not necessary
# you can skip this part and go to #7

import pandas as pd
from sklearn import mixture
import numpy as np 
import matplotlib.pyplot as plt
import plotly.express as px

data_all = pd.read_csv("D:/Google Drive/UVA_PhD/Projects/Structural Equation Modeling/data.csv")
data = data_all.drop(['Timestamp','DeviceId','DeviceId.1','video_name'],axis=1)

In [ ]:
#6. finding the best discreetization cluster number for gmm algorithmically not necessary
# you can skip this part and go to #7

for col in data.columns:
    print(col)
    temp = data[col]
    X = temp.values
    X = X.reshape(-1,1)
    n_components = np.arange(1, 15)
    models = [mixture.GaussianMixture(n, random_state=0).fit(X)
              for n in n_components]
    plt.plot(n_components, [m.bic(X) for m in models], label='BIC')
    plt.plot(n_components, [m.aic(X) for m in models], label='AIC')
    plt.legend(loc='best')
    plt.xlabel('n_components');
    plt.show()

In [ ]:
#7. putting the clusters manually as a dictionray and clustering them 

from sklearn import mixture
import matplotlib.pyplot as plt
import plotly.express as px
#data_all = total_data.copy()
components_modal = {' gaze_angle_x':2, ' gaze_angle_y':2,'mag_gaze':2}

for col in data_all.columns:
    if col in [' gaze_angle_x',' gaze_angle_y','mag_gaze']:
        print(col)
        temp = data_all[col]
        X = temp.values
        X = X.reshape(-1,1)
        labels = (mixture.GaussianMixture(n_components=components_modal[col]).fit(X)).predict(X)
        data_all[(col+str("_label"))] = labels

In [ ]:
#8. Adding road type info 

start_date = pd.Timestamp("2020-08-23T15:25:22.000+0000",tz=None)
end_date = pd.Timestamp("2020-08-23T17:16:00.000+0000",tz=None)
start_date = start_date.tz_localize(None)
end_date = end_date.tz_localize(None)
data_all.index = pd.to_datetime(data_all.index)
data_all["road_type"] = None
data_all.loc[data_all.index<start_date,'road_type']=0
data_all.loc[data_all.index>=start_date,'road_type']=1

In [ ]:
#9. Adding time factor

time_1 = pd.Timestamp("2020-08-23T15:25:21.000+0000",tz=None)
time_2 = pd.Timestamp("2020-08-23T16:21:21.000+0000",tz=None)
time_3 = pd.Timestamp("2020-08-23T17:21:21.000+0000",tz=None)
time_1= time_1.tz_localize(None)
time_2= time_2.tz_localize(None)
time_3= time_3.tz_localize(None)

data_all["time_factor"] = None
data_all.loc[data_all.index<time_2,'time_factor']=0
data_all.loc[data_all.index>=time_2,'time_factor']=1

In [ ]:
data_all.to_csv("data_everything.csv")

In [ ]:
#10.1 Same analysis for 03072021 trip - watch

#reading acc,gyro, and linear acc data, put them together, resample at 100 hz, fill nans and make it a clean dataframe
import pandas as pd 
from tqdm import tqdm
from scipy.signal import find_peaks,detrend

sensors = ["HeartRateDatum","GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum","PPGDatum"]
df_ultimate = []

for sensor in sensors:
    df_temp = pd.read_csv("C:/Users/Arsalan/Desktop/CVILLE - DC Trips/03072021 DC- CVille/"+"Smartwatch_"+sensor+".csv",parse_dates=['Timestamp'])
    start_date = "2021-03-05T20:31:21.000+0000"
    end_date = "2021-03-05T23:27:00.000+0000"
#     start_date = "2020-08-23T15:16:11.000+0000"
#     end_date = "2020-08-23T17:16:00.000+0000"
    mask = (df_temp['Timestamp'] > start_date) & (df_temp['Timestamp'] <= end_date)
    df_temp = df_temp.loc[mask]
    if sensor in ["GyroscopeDatum","AccelerometerDatum","LinearAccelerationDatum"]:
        df_temp.rename(columns={'X': 'X'+sensor[:4]+"_driver", 'Y': 'Y'+sensor[:4]+"_driver",'Z':'Z'+sensor[:4]+"_driver"}, inplace=True)
    df_ultimate.append(df_temp)

    
df_final = df_ultimate[0]
for df in df_ultimate[1:]:
    df = df.drop(columns=["participantid"])
    df.reset_index(drop=True,inplace=True)
    df_final = df_final.merge(df,on=['Timestamp',
                                         'DeviceId'], how='outer')
    
    
df_final = df_final.set_index("Timestamp")
df_final = df_final.groupby(["DeviceId"]).resample('10L').mean().ffill().bfill()
df_final = df_final.reset_index()
df_final["magGyro_driver"] = (df_final["XGyro_driver"]**2 + df_final["YGyro_driver"]**2 + df_final["ZGyro_driver"]**2)**0.5
df_final["magAcce_driver"] = (df_final["XAcce_driver"]**2 + df_final["YAcce_driver"]**2 + df_final["ZAcce_driver"]**2)**0.5
df_final["magLine_driver"] = (df_final["XLine_driver"]**2 + df_final["YLine_driver"]**2 + df_final["ZLine_driver"]**2)**0.5
df_final["magGyro_driver"] = (detrend(df_final["magGyro_driver"].values))**2
df_final["magAcce_driver"] = (detrend(df_final["magAcce_driver"].values))**2
df_final["magLine_driver"] = (detrend(df_final["magLine_driver"].values))**2

#keep a copy to not rerun things
hr_imu_data = df_final.copy()
hr_imu_data = hr_imu_data.set_index("Timestamp")
#imu_data_right = imu_data_right.set_index("Timestamp")
#imu_data_left = imu_data_left.set_index("Timestamp")

In [ ]:
#10.2 Same analysis for 03072021 trip - gaze
#check for time difference 

cols = [ " AU01_r"," AU02_r"," AU04_r"," AU05_r"," AU06_r"," AU07_r"," AU09_r"," AU10_r"," AU12_r"," AU14_r"," AU15_r"," AU17_r"," AU20_r"," AU23_r"," AU25_r"," AU26_r"," AU45_r"," AU01_c"," AU02_c"," AU04_c"," AU05_c"," AU06_c"," AU07_c"," AU09_c"," AU10_c"," AU12_c"," AU14_c"," AU15_c"," AU17_c"," AU20_c"," AU23_c"," AU25_c"," AU26_c"," AU28_c", " AU45_c"
," timestamp"," confidence"," success"," gaze_angle_x"," gaze_angle_y"," p_scale"," p_rx"," p_ry"," p_rz"," p_tx"," p_ty","video_name","day" ]
gaze = pd.read_csv("I:/Phase 1/9/Video/03072021/OpenFace/final_gaze.csv",usecols=cols)

gaze["time_start"] = (gaze["video_name"].str[:-7])
gaze["time_start"] = pd.to_datetime(gaze["time_start"],format = '%Y%m%d_%H%M%S')
gaze[" timestamp"] = pd.to_timedelta(gaze[" timestamp"],unit = 's')
gaze["Timestamp"] = gaze["time_start"] + gaze[" timestamp"]
gaze["mag_gaze"] = (gaze[" gaze_angle_x"]**2 + gaze[" gaze_angle_y"]**2)**0.5
gaze["Timestamp"] = gaze["Timestamp"] + pd.Timedelta(value=+5,unit='h')
start_date = pd.Timestamp("2021-03-05T20:31:21.000+0000",tz=None)
end_date = pd.Timestamp("2021-03-05T23:27:00.000+0000",tz=None)
#start_date = pd.Timestamp("2020-08-23T15:16:11.000+0000",tz=None)
#end_date = pd.Timestamp("2020-08-23T17:16:00.000+0000",tz=None)
start_date = start_date.tz_localize(None)
end_date = end_date.tz_localize(None)
gaze["Timestamp"] = pd.to_datetime(gaze["Timestamp"])  
gaze["Timestamp"] = gaze["Timestamp"].dt.tz_localize(None)
mask = (gaze["Timestamp"] > start_date) & (gaze["Timestamp"] <= end_date)
gaze = gaze.loc[mask]
gaze = gaze.set_index("Timestamp")
gaze = gaze[gaze[" success"] == 1]
gaze = gaze[gaze[" confidence"]>0.85]
gaze = gaze.resample('10L').mean().bfill().ffill()
gaze = gaze.drop(columns=['day'])

In [ ]:
#10.3 Same analysis for 03072021 trip - mask rcnn
#check for time difference 

import os
import pandas as pd

cols = ["person","bicycle","car","motorcycle","bus","truck","frame","follow_dist","follow_obj","lead_pres","video_name","day","traffic light" ]
mask_rcnn = pd.read_csv("D:/Google Drive/UVA_PhD/Projects/Structural Equation Modeling/csv files/final_mask_rcnn.csv",usecols=cols)

mask_rcnn["time_start"] = (mask_rcnn["video_name"].str[:-16])
mask_rcnn["time_start"] = pd.to_datetime(mask_rcnn["time_start"],format = '%Y%m%d_%H%M%S')
mask_rcnn[" timestamp"] = mask_rcnn["frame"]*0.0333
mask_rcnn[" timestamp"] = pd.to_timedelta(mask_rcnn[" timestamp"],unit = 's')
mask_rcnn["Timestamp"] = mask_rcnn["time_start"] + mask_rcnn[" timestamp"]
mask_rcnn["Timestamp"] = mask_rcnn["Timestamp"] + pd.Timedelta(value=+5,unit='h')
start_date = pd.Timestamp("2021-03-05T20:31:21.000+0000",tz=None)
end_date = pd.Timestamp("2021-03-05T23:27:00.000+0000",tz=None)
#start_date = pd.Timestamp("2020-08-23T15:16:11.000+0000",tz=None)
#end_date = pd.Timestamp("2020-08-23T17:16:00.000+0000",tz=None)
start_date = start_date.tz_localize(None)
end_date = end_date.tz_localize(None)
mask_rcnn["Timestamp"] = pd.to_datetime(mask_rcnn["Timestamp"])  
mask_rcnn["Timestamp"] = mask_rcnn["Timestamp"].dt.tz_localize(None)
mask = (mask_rcnn["Timestamp"] > start_date) & (mask_rcnn["Timestamp"] <= end_date)
mask_rcnn = mask_rcnn.loc[mask]
mask_rcnn = mask_rcnn.set_index("Timestamp")
mask_rcnn["lead_pres"] = mask_rcnn["lead_pres"].fillna(0)
mask_rcnn["follow_obj"] = mask_rcnn["follow_obj"].fillna(15)
mask_rcnn["follow_dist"] = mask_rcnn["follow_dist"].fillna(mask_rcnn["follow_dist"].max(), downcast='infer')
mask_rcnn["all_vehicles"] = mask_rcnn["car"] + mask_rcnn["truck"]
mask_rcnn = mask_rcnn.resample('10L').bfill().ffill()


In [ ]:
#10.4 Same analysis for 03072021 trip put them all together

hr_imu_data.index = hr_imu_data.index.tz_localize(None)
total_data = pd.concat([mask_rcnn,gaze,hr_imu_data],axis=1)

#4. combining them with change points - copying bcp groups from previous steps
total_data = total_data[:-1]
total_data = total_data.bfill().ffill()

In [ ]:
#10.5 combining with hav data - you can also get the total_data and run hav on it

features_needed = ['all_vehicles','traffic light','follow_dist',' gaze_angle_x',' gaze_angle_y',
                  ' AU01_c', ' AU02_c', ' AU04_c', ' AU05_c', ' AU06_c', ' AU07_c',
                   ' AU09_c', ' AU10_c', ' AU12_c', ' AU14_c', ' AU15_c', ' AU17_c',
                   ' AU20_c', ' AU23_c', ' AU25_c', ' AU26_c', ' AU28_c', ' AU45_c',
                  'PPG1','PPG3', 'HR']
total_data_cleaned = total_data[features_needed]

#if you need to save it run this 
#total_data_cleaned.to_csv("03072021_going_raw.csv")

total_data_cleaned = total_data_cleaned.reset_index()
hav_inc = pd.read_csv("csv files/hav_inc_cleaned_03072021.csv")
feat_list = ["PPG1 _hav","PPG3 _hav","gaze_angle_x _hav","gaze_angle_y _hav","follow_dist _hav"]
hav_inc = hav_inc[feat_list]
total = pd.concat([hav_inc,total_data_cleaned],axis=1)
total = total.dropna()
total = total.rename(columns={"PPG1 _hav":"PPG1_hav","PPG3 _hav":"PPG3_hav",
                              "gaze_angle_x _hav":"gaze_angle_x_hav",
                              "gaze_angle_y _hav":"gaze_angle_y_hav"
                             ,"traffic light":"traffic_light"})

total["PPG1_hav"] = total["PPG1_hav"]#**2
total["PPG3_hav"] = total["PPG3_hav"]#**2
total["gaze_angle_x_hav"] = total["gaze_angle_x_hav"]**2
total["gaze_angle_y_hav"] = total["gaze_angle_y_hav"]**2


total = total.set_index("Timestamp")
#total = total.resample("1000L").mean()
#total.to_csv("1s_resampled.csv")

In [ ]:
#11.0 Smoothing - exploration - high frequency

total["PPG1_hav_smooth"] = signal.savgol_filter(total["PPG1_hav"],499,2)
total["PPG3_hav_smooth"] = signal.savgol_filter(total["PPG3_hav"],499,2)
total["gaze_angle_x_hav_smooth"] = signal.savgol_filter(total["gaze_angle_x_hav"],499,2)
total["car_smooth"] = signal.savgol_filter(total["car"],499,2)
total["follow_dist_smooth"] = signal.savgol_filter(total["follow_dist"],499,2)
total["traffic_light_smooth"] = signal.savgol_filter(total["traffic_light"],499,2)
total = total.reset_index()

In [ ]:
#11.1 Smoothing - exploration

from scipy import signal

data_for_resample = pd.read_csv("1s_resampled_change_p_higher_p0.csv")
data_for_resample = data_for_resample.set_index("Timestamp")

#data_for_resample = total_resampled.resample('1S').mean()
data_for_resample["PPG1_hav_smooth"] = signal.savgol_filter(data_for_resample["PPG1_hav"],501,3)
data_for_resample["PPG3_hav_smooth"] = signal.savgol_filter(data_for_resample["PPG3_hav"],501,3)
data_for_resample["gaze_angle_x_hav_smooth"] = signal.savgol_filter(data_for_resample["gaze_angle_x_hav"],501,3)
data_for_resample["car_smooth"] = signal.savgol_filter(data_for_resample["all_vehicles"],2599,6)
data_for_resample["follow_dist_smooth"] = signal.savgol_filter(data_for_resample["follow_dist"],501,3)
data_for_resample["traffic_light_smooth"] = signal.savgol_filter(data_for_resample["traffic_light"],501,3)
data_for_resample["AU06_c_smooth"] = signal.savgol_filter(data_for_resample["AU06_c"],501,3)

In [ ]:
#11.2 Smoothing - real
from scipy import signal

data_for_resample = pd.read_csv("1s_resampled_change_p_higher_p0.csv")
data_for_resample = data_for_resample.set_index("Timestamp")
features = {'all_vehicles':121,'follow_dist':501,'traffic_light':501,'gaze_angle_x_hav':501,
       'gaze_angle_y_hav':501,'AU01_c':501, 'AU02_c':501, 
            'AU04_c':501, 'AU05_c':501, 'AU06_c':501, 'AU07_c':501,
       'AU09_c':501, 'AU10_c':501, 'AU12_c':501, 'AU14_c':501, 'AU15_c':501, 'AU17_c':501,
       'AU20_c':501, 'AU23_c':501, 'AU25_c':501, 'AU26_c':501, 'AU28_c':501, 'AU45_c':501, 'HR _bcp_prob':501,
           'PPG1_hav':501,'PPG3_hav':501}
for key in features:
    data_for_resample[str(key)+"_smooth"] = signal.savgol_filter(data_for_resample[key],features[key],2)
data_for_resample = data_for_resample.reset_index()
#data_for_resample.to_csv("all_data_1s_higher_p0.csv")

In [ ]:
data_for_peak = pd.read_csv("1s_resampled_change_p_higher_p0.csv",parse_dates=["Timestamp"])

In [ ]:
#Taking the rising change points

data_for_peak["Timestamp"] = data_for_peak["Timestamp"].dt.tz_localize(None)
data_for_peak["diff"] = data_for_peak["HR_bcp_mean"].diff()
data_for_peak["diff"] = data_for_peak["diff"].shift(-1)
data_for_peak["type_of_cp"] = data_for_peak["diff"].apply(lambda x: 1 if x>0.9 else 0)
data_for_peak["rising_cp"] = 0
data_for_peak["rising_cp"][(data_for_peak["HR_bcp_prob"]!=0)&(data_for_peak["type_of_cp"]==1)] = 1

In [ ]:
# resampel accelerometer data and combine

hr_imu_data_res = hr_imu_data.resample("1S").mean()
hr_imu_data_res = hr_imu_data_res.reset_index()
hr_imu_data_res["Timestamp"] = hr_imu_data_res["Timestamp"].dt.tz_localize(None)
data_with_acc = pd.merge(hr_imu_data_res,data_for_peak,on="Timestamp")
data_for_peak = data_with_acc.copy()

In [ ]:
from scipy import signal
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt

features = {'all_vehicles_bcp_prob':0.1,'follow_dist_bcp_prob':0.5,'traffic_light':3,
           'PPG1_hav':9.493063780879695e-07,'PPG3_hav':9.493065626152543e-07,"HR_bcp_prob":1,"AU06_c":0.1,"AU12_c":0.1
           ,"magGyro_driver":0.2,"magAcce_driver":1,"rising_cp":1}
features_dist = {'all_vehicles_bcp_prob':1,'follow_dist_bcp_prob':1,'traffic_light':20,
           'PPG1_hav':1,'PPG3_hav':1,"HR_bcp_prob":600,"AU06_c":1,"AU12_c":1,"magGyro_driver":1,"magAcce_driver":1,"rising_cp":600}

for feat in features:
    #plt.figure()
    x = data_for_peak[feat]
    peaks, _ = find_peaks(x, height=features[feat],distance=features_dist[feat])
    #sns.distplot(peaks[1:]-peaks[:-1])
    boolean_obs = np.zeros(x.shape[0])
    boolean_obs[peaks] = 1
    data_for_peak[feat+"boolean"] = boolean_obs
    data_for_peak[feat+"coin_prob"] = data_for_peak[feat+"boolean"].rolling(window=240,min_periods=0).mean()

In [ ]:
def invgauss_fit(bool_peaks, get='hr'):
    peaks = np.where(bool_peaks==1)[0]
    waiting_times = np.array(peaks[1:]-peaks[:-1])
    frequencies = 1.0/waiting_times
    mu = waiting_times.mean()
    #sns.distplot(frequencies)
    theta = 1.0/(((frequencies - 1.0/mu).sum())/frequencies.shape[0])
    
    mean_arrival_rate = 1.0/mu + 1.0/theta
    std_arrival_rate = np.sqrt(1.0/(mu*theta) + 2.0/(theta*theta))
    std_arrival_time = np.sqrt(mu**3/theta)
    
    if get == 'hr':
        return mean_arrival_rate
    elif get=='hrv':
        return std_arrival_rate
    
def inv1(bool_peaks):
    return invgauss_fit(bool_peaks, get='hr')

def inv2(bool_peaks):
    return invgauss_fit(bool_peaks, get='hrv')

In [ ]:
features = {'all_vehicles_bcp_prob':0.1,'follow_dist_bcp_prob':0.5,'traffic_light':1,
           'PPG1_hav':9.493063780879695e-07,'PPG3_hav':9.493065626152543e-07,"HR_bcp_prob":0.1,"AU06_c":0.1,"AU12_c":0.1,
            "magGyro_driver":0.2,"magAcce_driver":1}
for feat in features:
    data_for_peak[feat+"inv"] = data_for_peak[feat+"boolean"].rolling(window=240,min_periods=1).apply(inv1)

data_for_peak["traffic_light_moving"] = data_for_peak["traffic_lightboolean"].rolling(600,min_periods=0).mean()
data_for_peak = data_for_peak.bfill().ffill()

### Data from PPG feature Extraction - See PPG Analysis folder notebook

In [ ]:
#Nk PPG

import pandas as pd

PPG_features = pd.read_csv("All_features_PPG3.csv",parse_dates=["Timestamp"])
PPG_features = PPG_features.set_index("Timestamp")
PPG_features = PPG_features.resample("1S").mean()
PPG_features = PPG_features.reset_index()

In [ ]:
#Heartpy PPG
PPG_features = pd.read_csv("D:/Google Drive/UVA_PhD/Projects/PPG Analysis/PPG A 0307/final_heartpy_PPG.csv",parse_dates=["Timestamp"])
mask_rmssd = np.where(np.isclose(PPG_features['RMSSD'], 7777))
mask_breathing = np.where(np.isclose(PPG_features['breathing'], 7777))
PPG_features["RMSSD"].loc[mask_rmssd] = np.nan
PPG_features["breathing"].loc[mask_breathing] = np.nan
PPG_features = PPG_features.set_index("Timestamp")
PPG_features = PPG_features.resample("1S").mean()
PPG_features = PPG_features.reset_index()
PPG_features["Timestamp"] = PPG_features["Timestamp"].dt.tz_localize(None)

### Data from gaze feature Extraction - See gaze feature extraction folder notebook

In [ ]:
import pandas as pd

gaze_ent = pd.read_csv("D:/Google Drive/UVA_PhD/Projects/gaze feature extraction/gaze_ent_in.csv",parse_dates=["Timestamp"])
gaze_ent = gaze_ent.set_index(["Timestamp"])
gaze_ent = gaze_ent["entropy"]
gaze_ent = gaze_ent.resample("1S").mean()

In [ ]:
gaze_ent = gaze_ent.reset_index()
final_data = pd.merge(PPG_features,data_for_peak,on="Timestamp")
final_data = pd.merge(final_data,gaze_ent,on="Timestamp")
final_data = final_data.set_index("Timestamp")
final_data = final_data.reset_index()
final_data["PPG3_hav_moving"] = final_data["PPG3_hav"].rolling(240,min_periods=0).mean()
final_data["all_vehicles_moving"] = final_data["all_vehicles_bcp_prob"].rolling(240,min_periods=1).mean()
final_data["magGyro_driver_moving"] = final_data["magGyro_driverboolean"].rolling(240,min_periods=1).mean()
final_data["all_vehicles_moving"] = final_data["all_vehicles"].rolling(240,min_periods=1).mean()

### Move here and read the file below for saving time. 

In [ ]:
import pandas as pd
final_data = pd.read_csv("state_space_model_1.csv")

In [ ]:
#12. plotting the smooth

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy
from scipy import signal
import plotly.express as px
from plotly.subplots import make_subplots

#
data = final_data.copy()
fig1 = make_subplots(rows=3, cols=1, 
                    shared_xaxes=True,vertical_spacing =0.05,specs=[[{"secondary_y": True}],[{"secondary_y": True}]
                                                                   ,[{"secondary_y": True}]])
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["HR_bcp_mean"],name= "entropy",yaxis='y1',line=dict(width=1,color='black'))
               ,row=1,col=1)
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["rising_cpboolean"],name= "hrv_inv",yaxis='y2',line=dict(width=1,color='red'))
               ,row=1,col=1,secondary_y=True)
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["magGyro_driver_moving"],name= "Magnitude of Gyro",yaxis='y3',line=dict(width=1,color='black'))
               ,row=2,col=1)
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["entropy"],name= "Gaze Entropy",yaxis='y4',line=dict(width=1,color='red'))
               ,row=2,col=1,secondary_y=True)
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["all_vehicles_moving"],name= "No. of Vehicles",yaxis='y5',line=dict(width=1,color='black'))
               ,row=3,col=1)
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["entropy"],name= "Intersection",yaxis='y6',line=dict(width=1,color='red'))
               ,row=3,col=1,secondary_y=True)
# fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["all_vehicles_bcp_probinv"],name= "HR",yaxis='y1',line=dict(width=4,color='black'))
#                ,row=6,col=1)
# fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["follow_dist_bcp_probinv"],name= "Vehicles",yaxis='y1',line=dict(width=4,color='black'))
#                ,row=7,col=1)
# fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["traffic_light_moving"],name= "Follow_dist",yaxis='y1',line=dict(width=4,color='black'))
#                ,row=8,col=1)



fig1.update_layout(  
    {    
        'yaxis':{'title':"<b>entropy",'linecolor':'black'},
        'yaxis2':{'title':"<b>HRV - RMSSD",'linecolor':'black'},
        'yaxis3':{'title':"<b>Magnitude of Gyro ",'linecolor':'black'},
        'yaxis4':{'title':"<b>Gaze Entropy",'linecolor':'black'},
        'yaxis5':{'title':"<b>No. of Vehicles",'linecolor':'black'},
        'yaxis6':{'title':"<b>Intersection",'linecolor':'black'},
        'yaxis7':{'title':"<b>Follow_dist",'linecolor':'black'},
        'yaxis8':{'title':"<b>Traffic-light-moving",'linecolor':'black'},
        'height':1080,
        'width':1920,
        'font':dict(size=20),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)'
        
    }
)
fig1.update_layout(legend=dict(
    yanchor="top",
    xanchor="left",
    
))


fig1.show()


In [ ]:
final_data.to_csv("state_space_model_spaced_change_points.csv")

### Space change points in a different way with min+sd

In [ ]:
import pandas as pd

data_for_peak = pd.read_csv("state_space_model_1.csv",parse_dates=["Timestamp"])

In [ ]:
data_for_peak["Timestamp"] = data_for_peak["Timestamp"].dt.tz_localize(None)
data_for_peak["diff"] = data_for_peak["HR_bcp_mean"].diff()
data_for_peak["diff"] = data_for_peak["diff"].shift(-1)
data_for_peak["type_of_cp"] = data_for_peak["diff"].apply(lambda x: 1 if x>(0) else 0)
data_for_peak["rising_cp"] = 0
data_for_peak["rising_cp"][(data_for_peak["HR_bcp_prob"]!=0)&(data_for_peak["type_of_cp"]==1)] = 1

In [ ]:
from scipy import signal
from scipy.signal import find_peaks


x = data_for_peak["rising_cp"]
peaks, _ = find_peaks(x, height=1,distance=300)
#sns.distplot(peaks[1:]-peaks[:-1])
boolean_obs = np.zeros(x.shape[0])
boolean_obs[peaks] = 1
data_for_peak["rising_cp"+"boolean"] = boolean_obs
data_for_peak["rising_cp"+"coin_prob"] = data_for_peak["rising_cp"+"boolean"].rolling(window=240,min_periods=0).mean()

In [ ]:
def above_sd(segment):
    if segment>(2*data_for_peak["HR_x"].std()+data_for_peak["HR_x"].mean()):
        return 1
    else:
        return 0
data_for_peak["above_sd"] = data_for_peak["HR_bcp_mean"].apply(above_sd)

In [ ]:
def find_any_peak(segment):    
    if ((segment > 0.1)).any():
        return 1
    else:
        return 0


#data_all_speed_19["overlapping"] = 
data_for_peak["above_sd_presence"] = data_for_peak["above_sd"].rolling(3).apply(find_any_peak).dropna()
data_for_peak["rising_cp_presence"] = data_for_peak["rising_cp"].rolling(3).apply(find_any_peak).dropna()
data_for_peak["abnormal_loc"] =data_for_peak["above_sd_presence"] + data_for_peak["rising_cp_presence"]
data_for_peak["abnormal_loc"][data_for_peak["abnormal_loc"]!=2] =0
data_for_peak["abnormal_loc"][data_for_peak["abnormal_loc"]==2] =1

In [ ]:
#12. plotting the smooth

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy
from scipy import signal
import plotly.express as px
from plotly.subplots import make_subplots

#
data = data_for_peak.copy()
fig1 = make_subplots(rows=1, cols=1, 
                    shared_xaxes=True,vertical_spacing =0.05,specs=[[{"secondary_y": True}]
                                                                   ])
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["HR_bcp_mean"],name= "Heart Rate",yaxis='y1',line=dict(width=1,color='black'))
               ,row=1,col=1)
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["abnormal_loc"],name= "Segments",yaxis='y2',line=dict(width=1,color='red'))
               ,row=1,col=1,secondary_y=True)
# fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["abnormal_loc"],name= "Change Points",yaxis='y2',line=dict(width=1,color='red'))
#                ,row=2,col=1)
# fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["above_sd"],name= "Change Points",yaxis='y2',line=dict(width=1,color='red'))
#                ,row=3,col=1)



fig1.update_layout(  
    {    
        'xaxis':{'title':"<b>Time",'linecolor':'black'},
        'yaxis':{'title':"<b>HR",'linecolor':'black'},
        'yaxis2':{'title':"<b>Change Points",'linecolor':'black'},
        'height':400,
        'width':1200,
        'font':dict(size=20),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)'
        
    }
)
fig1.update_layout(legend=dict(
    yanchor="top",
    xanchor="left",
    
))


fig1.show()


In [ ]:
data_for_peak.to_csv("state_space_model_spaced_change_point_sd.csv")

### Plotting segments together with their State-space results

In [ ]:
import pandas as pd
import numpy as np

data_for_res = pd.read_csv("state_space_model_spaced_change_point_sd.csv")
results_of_state_space = pd.read_csv("change_point_based_state_space.csv")
results_of_state_space = results_of_state_space.dropna()
for index,row in results_of_state_space.iterrows():
    for col in results_of_state_space.columns:
        data_for_res.loc[row["start"]:row["end"],col] =row[col]

In [ ]:
#12. plotting the smooth

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy
from scipy import signal
import plotly.express as px
from plotly.subplots import make_subplots

#
data = data_for_res.copy()
fig1 = make_subplots(rows=2, cols=1, 
                    shared_xaxes=True,vertical_spacing =0.05,specs=[[{"secondary_y": True}],[{"secondary_y": True}]
                                                                   ])
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["HR_x"],name= "Heart Rate",yaxis='y1',line=dict(width=1,color='black'))
               ,row=1,col=1)
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["abnormal_loc"],name= "Segments",yaxis='y2',line=dict(width=1,dash="dash",color='red'))
               ,row=1,col=1,secondary_y=True)
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["B.b1"],name= "Stress Transition",yaxis='y3',line=dict(width=4,color='blue'))
               ,row=2,col=1)
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["B.b4"],name= "Work Load Transition",yaxis='y4',line=dict(width=4,color='green'))
               ,row=2,col=1,secondary_y=True)



fig1.update_layout(  
    {    
        
        'yaxis':{'title':"<b>HR",'linecolor':'black'},
        'yaxis2':{'title':"<b>Change Points",'linecolor':'black'},
        'yaxis3':{'title':"<b>Transition Coefficient",'linecolor':'black'},
        'yaxis4':{'title':"<b>Transition Coefficient",'linecolor':'black'},
        'xaxis2':{'title':"<b>Time",'linecolor':'black'},
        'height':1100,
        'width':1820,
        'font':dict(size=20),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)'
        
    }
)
fig1.update_layout(legend=dict(
    yanchor="top",
    xanchor="left",
    orientation="h",
    x=0,
    y=1.09
    
))


fig1.show()


### Read the segmented state space results

In [ ]:
import pandas as pd

results_state_space = pd.read_csv("results_state_space_10.csv")
results_state_space["file_number"] = results_state_space["name"].str.split("_",n=1).str[0]
results_state_space["file_number"] = pd.to_numeric(results_state_space["file_number"])
results_state_space = results_state_space.sort_values(by=["file_number"])
results_state_space = results_state_space[results_state_space['errors']==0]


data_for_res = pd.read_csv("state_space_model_spaced_change_point_sd.csv",parse_dates=["Timestamp"])
for index,row in results_state_space.iterrows():
    for col in results_state_space.columns:
        data_for_res.loc[row["file_number"]:row["file_number"]+10,col] =row[col]

In [ ]:
data_for_res['cross_corr'] = data_for_res['B.b1'].rolling(400,min_periods=0).corr(data_for_res['B.b4'])

In [ ]:
data_for_res = data_for_res.dropna(subset=['file_number'])

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

def crosscorr(datax, datay, lag=0, wrap=False):
    """ Lag-N cross correlation. 
    Shifted data filled with NaNs 
    
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length
    Returns
    ----------
    crosscorr : float
    """
    if wrap:
        shiftedy = datay.shift(lag)
        shiftedy.iloc[:lag] = datay.iloc[-lag:].values
        return datax.corr(shiftedy)
    else: 
        return datax.corr(datay.shift(lag))

In [ ]:
def time_lagged_cross_corr(sigA,sigB,window_size=120,step_size=1):
    t_start = 0
    t_end = t_start + window_size
    rss=[]
    while t_end < sigA.shape[0]:
        d1 = sigA.iloc[t_start:t_end]
        d2 = sigB.iloc[t_start:t_end]
        rs = [crosscorr(d1,d2, lag, wrap=False) for lag in range(-200,210,10)]
        rss.append(rs)
        t_start = t_start + step_size
        t_end = t_end + step_size
    rss = pd.DataFrame(rss,columns=range(-200,210,10))
    
    return rss.idxmax(axis=1)

data_for_res['lag_val'] = 0
data_for_res['lag_val'][240:] = time_lagged_cross_corr(data_for_res['B.b1'],data_for_res['B.b4'],240,1)    
    

In [ ]:
def crosscorr(datax, datay, window_size, lag=0):
    """ Lag-N cross correlation. 
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length

    Returns
    ----------
    crosscorr : float
    """
    return datax.rolling(window_size, center=True).corr(datay.shift(lag))


WCC = np.array([crosscorr(data_for_res['B.b1'], data_for_res['B.b4'], window_size=120, lag=i) for i in range(-200,200+1)])


In [ ]:
from scipy.signal import argrelextrema

def WCC_lag(WCC):
    center = int(np.floor(WCC.shape[0]/2))
    right = WCC[center:,]
    left = WCC[:center,]
    lag_lists = []
    for i in range(right.shape[1]):
        right_temp = right[:,i]
        #print(right_temp)
        left_temp = left[:,i]
        
        try:
            lag_right = argrelextrema(right_temp, np.greater)[0][0]
        except:
            try:
                if (np.argmax(right_temp)==right_temp.shape[0]-1) or (np.argmax(right_temp)==0):
                    lag_right = np.argmax(right_temp)
            except:
                lag_right = np.inf
        try:
            lag_left = argrelextrema(left_temp, np.greater)[0][-1] - center
        except:
            try:
                if (np.argmax(left_temp)==left_temp.shape[0]-1) or (np.argmax(left_temp)==0):
                    lag_left = np.argmax(left_temp)
            except:
                lag_left = np.inf
        lags = np.array([lag_left,lag_right])
        #print(lags)
        min_lag = lags[np.argmin(abs(lags))]#min(abs(lag_right),abs(lag_left))
        lag_lists.append(min_lag)
    return lag_lists
WCC_output = WCC_lag(WCC)

In [ ]:
WCC_output = pd.DataFrame(np.array(WCC_output))
data_for_res['lag_val'] = WCC_output

In [ ]:
# bring and clean speed data

speed_0305 = pd.read_csv("I:/Phase 1/9/Video/03072021/speed_csv/final_speed.csv")
speed_0305["Timestamp"] = speed_0305['Date'].str.cat(speed_0305['Time'],sep="T")
speed_0305["Timestamp"] = pd.to_datetime(speed_0305["Timestamp"], format="%m/%d/%YT%H:%M:%S")
speed_0305 = speed_0305.sort_values(by="Timestamp")
speed_0305["Timestamp"] = speed_0305["Timestamp"].dt.tz_localize(None)
speed_0305["Normalized Speed"] = speed_0305["Normalized Speed"].replace(to_replace="not available",method='bfill') 
speed_0305["Timestamp"] = speed_0305["Timestamp"] + pd.Timedelta(value=+4,unit='h')
start_date = pd.Timestamp("2021-03-05T20:35:20.000+0000",tz=None)
end_date = pd.Timestamp("2021-03-05T23:26:54.000+0000",tz=None)
start_date = start_date.tz_localize(None)
end_date = end_date.tz_localize(None)
mask = (speed_0305['Timestamp'] > start_date) & (speed_0305['Timestamp'] <= end_date)
speed_0305 = speed_0305.loc[mask]
#speed_for_bcp = total_data.resample('1S').mean().bfill().ffill()
#speed_for_bcp = data_for_bcp.reset_index()

speed_0305.to_csv("cleaned_speed_0305_for_bcp.csv")

In [ ]:
# perform bcp and bring the file back

speed_0305_bcp = pd.read_csv("D:/Google Drive/UVA_PhD/Projects/Structural Equation Modeling/cleaned_speed_0305_added_bcp.csv",parse_dates=["Timestamp"])

In [ ]:
# retreive max lag based steve's method

lags_steve = pd.read_csv("D:/Google Drive/UVA_PhD/Projects/Structural Equation Modeling/codeforwcc/results.csv")
data_for_res["lag_val_steve"]=np.nan
data_for_res['lag_val_steve'][-7822:] =lags_steve["V1"]

In [ ]:
#12. plotting the smooth

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy
from scipy import signal
import plotly.express as px
from plotly.subplots import make_subplots

#
data = data_for_res.copy()
fig1 = make_subplots(rows=3, cols=1, 
                    shared_xaxes=True,vertical_spacing =0.05,specs=[[{"secondary_y": True}],[{"secondary_y": True}],
                                                                    [{"secondary_y": True}]
                                                                   ])
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["B.b1"],name= "Stress Transition",yaxis='y1',line=dict(width=1,color='black'))
               ,row=1,col=1)
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["B.b4"],name= "Work Load Transition",yaxis='y2',line=dict(width=1,dash="dash",color='red'))
               ,row=1,col=1,secondary_y=True)
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["cross_corr"],name= "Cross Correlation",yaxis='y3',mode='markers')
               ,row=2,col=1)
fig1.add_trace(go.Scatter(x=data["Timestamp"], y=data["lag_val_steve"],name= "Highest Lag",yaxis='y4',mode='markers')
               ,row=2,col=1,secondary_y=True)
fig1.add_trace(go.Scatter(x=speed_0305_bcp["Timestamp"], y=speed_0305_bcp["Normalized.Speed"],name= "Speed",yaxis='y4')
               ,row=3,col=1)
fig1.add_trace(go.Scatter(x=speed_0305_bcp["Timestamp"], y=speed_0305_bcp["bcp_prob_speed"],name= "BCP Speed",yaxis='y4')
               ,row=3,col=1,secondary_y=True)

fig1.update_layout(  
    {    
        
        'yaxis1':{'title':"<b>Stress Transition",'linecolor':'black'},
        'yaxis2':{'title':"<b>Work Load Transition",'linecolor':'black'},
        'yaxis3':{'title':"<b>Cross Correlation",'linecolor':'black'},
        'yaxis4':{'title':"<b>Highest Lag",'linecolor':'black'},
        'yaxis5':{'title':"<b>Speed",'linecolor':'black'},
        'yaxis6':{'title':"<b>BCP Speed",'linecolor':'black'},
        'xaxis3':{'title':"<b>Time",'linecolor':'black'},
        'height':1300,
        'width':1000,
        'font':dict(size=20),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)'
        
    }
 )
fig1.update_layout(legend=dict(
    yanchor="top",
    xanchor="left",
    orientation="h",
    x=0,
    y=1.09
    
))


fig1.show()


### Plotting for the purpose of Paper

In [ ]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

data = final_data.copy()
list_of_feats=["Timestamp","HRV_RMSSD","entropy","all_vehicles_moving","magGyro_driver_moving","traffic_light_moving"]
data=data[list_of_feats]
data=data.set_index(["Timestamp"])
# create a scaler object
scaler = MinMaxScaler()
# fit and transform the data
df_norm = pd.DataFrame(scaler.fit_transform(data), columns=data.columns)
df_norm

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import wasserstein_distance
import plotly.graph_objects as go
import plotly.figure_factory as ff
import scipy
from scipy import signal
import plotly.express as px
from plotly.subplots import make_subplots

#
fig1 = make_subplots(rows=1, cols=1, 
                    shared_xaxes=True,vertical_spacing =0.05)
fig1.add_trace(go.Scatter(x=data.index, y=df_norm["HRV_RMSSD"],name= "RMSSD",yaxis='y1',line=dict(width=1,color='black'))
               ,row=1,col=1)
fig1.add_trace(go.Scatter(x=data.index, y=df_norm["entropy"],name= "Gaze Entropy",yaxis='y1',line=dict(width=1,color='red'))
               ,row=1,col=1)
fig1.add_trace(go.Scatter(x=data.index, y=df_norm["all_vehicles_moving"],name= "No. of Vehicles",yaxis='y1',line=dict(width=1,color='blue'))
               ,row=1,col=1)
fig1.add_trace(go.Scatter(x=data.index, y=df_norm["magGyro_driver_moving"],name= "Hand Movement",yaxis='y1',line=dict(width=1,color='orange'))
               ,row=1,col=1)
fig1.add_trace(go.Scatter(x=data.index, y=df_norm["traffic_light_moving"],name= "No. of Traffic Lights",yaxis='y1',line=dict(width=1,color='green'))
               ,row=1,col=1)



fig1.update_layout(  
    {   
        'xaxis':{'title':"<b>Time",'linecolor':'black'},
        'yaxis':{'title':"<b>Normalized Variable",'linecolor':'black'},
        'height':700,
        'width':1920,
        'font':dict(size=20),
        'paper_bgcolor':'rgba(0,0,0,0)',
        'plot_bgcolor':'rgba(0,0,0,0)'
        
    }
)
fig1.update_layout(legend=dict(
    yanchor="top",
    xanchor="left",
    
))


fig1.show()


### For visualizing peaks you can run the code below

In [ ]:
#import plotly.tools as tls
from scipy.signal import find_peaks,detrend
import numpy as np
import matplotlib.pyplot as plt

dat = data_for_peak.copy()
x = dat["all_vehicle_bcp_prob"]
peaks, _ = find_peaks(x, height=1)
boolean_obs = np.zeros(x.shape[0])
boolean_obs[peaks] = 1
fig, ax = plt.subplots(figsize=(20, 8))
#df_new['phasic'].plot()a
ax.plot(x.values)
ax.plot(peaks, x[peaks], "x",color="gray")
ax.plot(np.zeros_like(x), "--", color="gray")
#plt.ylim([8.7,11])
dat['boolean'] = boolean_obs
#tls.mpl_to_plotly(fig)
dat['coinprob'] = dat['boolean'].rolling(window=240).mean()